| 编号 |      魔术方法      |                             说明                             |
| :--: | :----------------: | :----------------------------------------------------------: |
|  1   |   \_\_getattr__    | 只有当读取的属性不存在的时候才会调用，默认会抛出AttributeError |
|  2   | \_\_getattribute__ |      无论属性是否存在都会调用，使用时要注意是否无限递归      |
|  3   |   \_\_setattr__    |                        设置属性时调用                        |
|  4   |   \_\_getattr__    |                        读取属性时调用                        |
|  5   |   \_\_delattr__    |                        del属性时调用                         |
|  6   |     \_\_dir__      |                 客制化dir(object)返回的内容                  |
|  7   |    \_\_slots__     |             变量，白名单存储object中可以有的属性             |

In [5]:
class AttributeMethod:
    def __init__(self):
        self.exist = "exist"

    def __getattr__(self, name):
        """
        只有当读取的属性不存在的时候才会调用
        默认会抛出AttributeError
        """
        print(f"__getattr__: {name}")
        return None
    
    def __getattribute__(self, name):
        """只要调用了属性就会调用这个函数，无论属性是否存在"""
        print(f"__getattribute__: {name}")
        # 一定要使用这样的写法，其他写法可能造成无限递归
        return super().__getattribute__(name)
    


In [6]:
# __getattr__和__getattribute__
# 
# 只要调用了属性不管存不存在都会调用__getattribute__
# 但如果属性不存在还会调用__getattr__

a = AttributeMethod()
print(a.exist)
print(a.test)

__getattribute__: exist
exist
__getattribute__: test
__getattr__: test
None


In [7]:
# 可以通过__getattribute__实现对属性调用次数进行统计
class StaticData:
    def __init__(self):
        self.data = "data"
        self.count = 0

    def __getattribute__(self, name):
        if name == "data":
            # 要注意的是，这里读取了count，其实也调用了一次__getattribute__，如果不加判断就会导致无限递归
            self.count += 1
        return super().__getattribute__(name)

s = StaticData()    
print(s.data)
print(s.data)
print(s.count)

data
data
2


***

In [8]:
# __setattr__实现所有实例都挂载到类变量上

class SetAttrMethod:
    _attr = {}
    def __init__(self) -> None:
        self.data = "abc"
    
    def __setattr__(self, name, value):
        """
        当设置属性值时调用此函数
        使用默认方法则return super().__setattr__(name,value)

        这里是将所有实例设置值时全部挂载到类变量上
        """
        self._attr[name] = value

    def __getattr__(self, name):
        """当读取实例属性的时候，返回村村储存在类变量中的value"""
        if name not in self._attr:
            raise AttributeError
        return self._attr[name]
    
a1 = SetAttrMethod()
a2 = SetAttrMethod()
a1.data = "123"
print(a2.data)
        


123


***

In [12]:
# ____delattr____当使用del删除属性时调用
class DelAttrMethod:
    def __init__(self):
        self.data = "abc"

    def __delattr__(self, name):
        print("delete attr:", name)
        super().__delattr__(name)
    
    def __getattr__(self, name):
        print(f"{name} is not exist")
        return None

d = DelAttrMethod()
# 触发__delattr__
del d.data
# data属性已经被删去
print(d.data)

delete attr: data
data is not exist
None


***

In [13]:
# dir(object)可以查看object中可以使用的方法和变量
# __dir__可以自定义返回内容
# 但是返回内容必须是一个sequence

class DirMethod:
    def __init__(self) -> None:
        self.data = "abc"
    
    def __dir__(self):
        """将返回内容删去魔术方法"""
        lst = super().__dir__()
        return [x for x in lst if not x.startswith("__")]
    
d = DirMethod()
print(dir(d))

['data']


***

描述器

In [17]:
class A:
    def __init__(self) -> None:
        self.val = 0

    def __get__(self, instance, owner=None):
        return self.val
    
    def __set__(self, instance, value):
        self.val = value

    def __delete__(self, instance):
        print("delete")

class B:
    # 这样的写法就是描述器
    x = A()

o = B()
print(o.x)

o.x = 1
o2 = B()
print(o2.x)


0
1


***

In [18]:
# __slots__白名单机制
# 允许class的object只能出现的属性
class A:
    __slots__ = ("x", "y")

a = A()
a.x = 1
a.z = 1


AttributeError: 'A' object has no attribute 'z'